In [1]:
# note- matches at lucene word level- if word itself gets changed, will most likely
# not find the match
import sodaclient

In [2]:
# Establish a connection to the soda web client
client = sodaclient.SodaClient("http://localhost:8080/soda")

In [3]:
# check the status of soda
ind_resp = client.index()
print(ind_resp)

{'status': 'ok', 'message': 'SoDA accepting requests (Solr version 7.3.0)'}


In [4]:
# list the lexicons
lex_resp = client.dicts()
print(lex_resp)

{'status': 'ok', 'lexicons': [{'lexicon': 'companies_addr', 'count': 1000}, {'lexicon': 'companies_name', 'count': 1000}, {'lexicon': 'companies_city', 'count': 1}, {'lexicon': 'companies_zip', 'count': 1}]}


In [22]:
# add dictionaries- alread did this, so do not need to run it in here
# add_resp = client.add(lexicon_name, id, names, commit)

In [18]:
# Annotate
user_entry_name = input("Enter the company name: ")
user_entry_addr = input("Enter the company address: ")

Enter the company name1 Mobile limited
Enter the company addr30 city road London


In [27]:
# check for lower case match
annot_resp = client.annot('companies_name', user_entry_name, 'lower')
print(annot_resp)

{'status': 'ok', 'annotations': [{'id': 'NAME_1', 'lexicon': 'companies_name', 'begin': 0, 'end': 16, 'coveredText': '1 Mobile limited', 'confidence': 1.0}]}


In [30]:
# Annotate
user_entry_name = "1 MOBILEs LimITed" #Original is "1 MOBILE LIMITED"
print(client.annot('companies_name', user_entry_name, 'stem1'))

{'status': 'ok', 'annotations': [{'id': 'NAME_1', 'lexicon': 'companies_name', 'begin': 0, 'end': 17, 'coveredText': '1 MOBILEs LimITed', 'confidence': 0.9411764705882353}]}


In [29]:
name1 = "BULL STUDIO LTD" #ORIGINAL IS "THE BULL STUDIO LTD"
print(client.annot('companies_name', name1, 'stop'))

{'status': 'ok', 'annotations': [{'id': 'NAME_645', 'lexicon': 'companies_name', 'begin': 0, 'end': 15, 'coveredText': 'BULL STUDIO LTD', 'confidence': 0.7894736842105263}]}


In [38]:
name2 = "Bull stúdios" #ORIGINAL IS "THE BULL STUDIO LTD"
print(client.rlookup('companies_name', name2, 'stem2'))

{'status': 'ok', 'entries': [{'id': 'NAME_645', 'lexicon': 'companies_name', 'text': 'THE BULL STUDIO LTD', 'confidence': 0.5263157894736842}]}


In [49]:
name2 = "Bull studio ltd." #ORIGINAL IS "THE BULL STUDIO LTD"
print(client.rlookup('companies_name', name2, 'stem3'))

{'status': 'ok', 'entries': [{'id': 'NAME_645', 'lexicon': 'companies_name', 'text': 'THE BULL STUDIO LTD', 'confidence': 0.7368421052631579}]}


In [50]:
addr1 = "30 city road"
print(client.rlookup('companies_addr', addr1, 'stem3'))

{'status': 'ok', 'entries': [{'id': 'ADR_538', 'lexicon': 'companies_addr', 'text': '30 CITY ROAD LONDON EC1Y 2AB', 'confidence': 0.42857142857142855}, {'id': 'ADR_7', 'lexicon': 'companies_addr', 'text': '30 CITY ROAD LONDON EC1 2AB', 'confidence': 0.4444444444444444}, {'id': 'ADR_461', 'lexicon': 'companies_addr', 'text': '30 CITY ROAD LONDON EC1Y 2AB', 'confidence': 0.42857142857142855}, {'id': 'ADR_155', 'lexicon': 'companies_addr', 'text': '30 CITY ROAD LONDON EC1Y 2AB', 'confidence': 0.42857142857142855}, {'id': 'ADR_313', 'lexicon': 'companies_addr', 'text': '30 CITY ROAD LONDON EC1Y 2AB', 'confidence': 0.42857142857142855}, {'id': 'ADR_322', 'lexicon': 'companies_addr', 'text': '30 CITY ROAD LONDON EC1Y 2AB', 'confidence': 0.42857142857142855}, {'id': 'ADR_202', 'lexicon': 'companies_addr', 'text': '30 CITY ROAD LONDON EC1Y 2AB', 'confidence': 0.42857142857142855}, {'id': 'ADR_203', 'lexicon': 'companies_addr', 'text': '30 CITY ROAD LONDON EC1Y 2AB', 'confidence': 0.42857142857

In [52]:
# format
user_entry_name = input("Enter the company name: ")
user_entry_addr = input("Enter the company address: ")
resp_name = client.rlookup('companies_name', user_entry_name, 'stem1')
resp_addr = client.rlookup('companies_addr', user_entry_addr, 'stem1')

Enter the company name: 1 Mobile limited
Enter the company address: 30 city road


In [72]:
# case 1- there is only one match in the name dictionary
if len(resp_name['entries']) == 1:
    name_id = resp_name['entries'][0]['id']
    addr_id = "ADR_" + resp_name['entries'][0]['id'].split('_')[1]
    #First, check if there is a matching address with the same ID
    for entry in resp_addr['entries']:
        if entry['id'] == addr_id:
            # print the full name and address to recommend to user
            print(resp_name['entries'][0]['text'] + " " + entry['text'])

{'id': 'ADR_1', 'lexicon': 'companies_addr', 'text': '30 CITY ROAD LONDON EC1Y 2AB', 'confidence': 0.42857142857142855}
1 MOBILE LIMITED 30 CITY ROAD LONDON EC1Y 2AB


In [75]:
# case 2- there are multiple matches in the name dictionary
# loop through both to find corresponding IDs
for name in resp_name['entries']:
    name_id = name['id']
    addr_id = "ADR_" + name_id.split('_')[1]
    for addr in resp_addr['entries']:
        if addr['id'] == addr_id:
            print("MATCH", name, addr)

MATCH {'id': 'NAME_1', 'lexicon': 'companies_name', 'text': '1 MOBILE LIMITED', 'confidence': 1.0} {'id': 'ADR_1', 'lexicon': 'companies_addr', 'text': '30 CITY ROAD LONDON EC1Y 2AB', 'confidence': 0.42857142857142855}


more likely the user will correctly enter the name- so partition this first
if there is 1 match with the name -> search for that id in the addr dict
if there are multiple matches with name -> search for any exact id matches in the two dictionaries

in case 2- also want to calculate the edit distance for the combined final address to rank them and provide those to the user

In [76]:
# calc edit distance of full user entry against the full match found

In [31]:
full_addr = "1 MOBILE LTD 30 city road london"
print(client.annot('companies_name', full_addr, 'stem1'))

{'status': 'ok', 'annotations': [{'id': 'NAME_1', 'lexicon': 'companies_name', 'begin': 0, 'end': 12, 'coveredText': '1 MOBILE LTD', 'confidence': 1.0}]}


In [32]:
print(client.annot('companies_addr', full_addr, 'stem1'))

{'status': 'ok', 'annotations': [{'id': 'ADDR_538', 'lexicon': 'companies_addr', 'begin': 13, 'end': 25, 'coveredText': '30 city road', 'confidence': 1.0}, {'id': 'ADDR_7', 'lexicon': 'companies_addr', 'begin': 13, 'end': 25, 'coveredText': '30 city road', 'confidence': 1.0}, {'id': 'ADDR_461', 'lexicon': 'companies_addr', 'begin': 13, 'end': 25, 'coveredText': '30 city road', 'confidence': 1.0}, {'id': 'ADDR_155', 'lexicon': 'companies_addr', 'begin': 13, 'end': 25, 'coveredText': '30 city road', 'confidence': 1.0}, {'id': 'ADDR_313', 'lexicon': 'companies_addr', 'begin': 13, 'end': 25, 'coveredText': '30 city road', 'confidence': 1.0}, {'id': 'ADDR_322', 'lexicon': 'companies_addr', 'begin': 13, 'end': 25, 'coveredText': '30 city road', 'confidence': 1.0}, {'id': 'ADDR_202', 'lexicon': 'companies_addr', 'begin': 13, 'end': 25, 'coveredText': '30 city road', 'confidence': 1.0}, {'id': 'ADDR_203', 'lexicon': 'companies_addr', 'begin': 13, 'end': 25, 'coveredText': '30 city road', 'conf

In [37]:
print(client.annot('companies_city1', full_addr, 'stem1'))

{'status': 'ok', 'annotations': [{'id': 'CITY_202', 'lexicon': 'companies_city1', 'begin': 26, 'end': 32, 'coveredText': 'london', 'confidence': 1.0}, {'id': 'CITY_203', 'lexicon': 'companies_city1', 'begin': 26, 'end': 32, 'coveredText': 'london', 'confidence': 1.0}, {'id': 'CITY_204', 'lexicon': 'companies_city1', 'begin': 26, 'end': 32, 'coveredText': 'london', 'confidence': 1.0}, {'id': 'CITY_205', 'lexicon': 'companies_city1', 'begin': 26, 'end': 32, 'coveredText': 'london', 'confidence': 1.0}, {'id': 'CITY_206', 'lexicon': 'companies_city1', 'begin': 26, 'end': 32, 'coveredText': 'london', 'confidence': 1.0}, {'id': 'CITY_207', 'lexicon': 'companies_city1', 'begin': 26, 'end': 32, 'coveredText': 'london', 'confidence': 1.0}, {'id': 'CITY_208', 'lexicon': 'companies_city1', 'begin': 26, 'end': 32, 'coveredText': 'london', 'confidence': 1.0}, {'id': 'CITY_209', 'lexicon': 'companies_city1', 'begin': 26, 'end': 32, 'coveredText': 'london', 'confidence': 1.0}, {'id': 'CITY_210', 'lex

In [27]:
print(client.annot('companies_zip', full_addr, 'stem1'))

{'status': 'ok', 'annotations': []}


In [29]:
print(client.annot('companies_ctry', "BLUESHED LIMITED", 'stem1'))

{'status': 'ok', 'annotations': []}


In [18]:
resp = client.delete('companies_name', "*")
resp1 = client.delete('companies_addr', "*")
resp2 = client.delete('companies_city', "*")
resp3 = client.delete('companies_zip', "*")

In [20]:
print(client.dicts())

{'status': 'ok', 'lexicons': [{'lexicon': 'companies_city1', 'count': 1000}, {'lexicon': 'companies_code', 'count': 1000}, {'lexicon': 'companies_ctry', 'count': 1000}, {'lexicon': 'companies_name', 'count': 1000}, {'lexicon': 'companies_addr', 'count': 939}]}


In [ ]:
# annot- takes a long string and finds substring matches in the dictionary
# rlookup- takes a short string that is missing info and finds matches in the dictionary